In [1]:
%cd ../..

c:\Users\aldion\Desktop\Development\hyperparameter-tuning-keras-tuner


In [2]:
import keras_tuner as kt
from tensorflow import keras

from src.utils.baseline_model import build_baseline_model
from src.utils.evaluator import Evaluator
from src.utils.history import TimeRecorder
from src.utils.hypermodel import model_builder
from src.utils.hyperparams import FixedHyperparams
from src.utils.load_data import load_data

### Load and Preprocess Data

In [3]:
data = load_data()
data.X_train.max(), data.X_test.max()

(255, 255)

In [4]:
data.preprocess()
data.X_train.max(), data.X_test.max()

(1.0, 1.0)

In [5]:
data.X_train.mean(), data.X_test.mean()

(0.2860405969887955, 0.2868492807122849)

### Baseline Model

In [6]:
model = build_baseline_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Baseline Model Training

In [7]:
hyper_parameters = FixedHyperparams(epochs=10, validation_split=0.2)
hyper_parameters

FixedHyperparams(epochs=2, validation_split=0.2)

In [8]:
epoch_time_recorder = TimeRecorder()

In [9]:
history = model.fit(data.X_train, data.y_train, epochs=hyper_parameters.epochs, 
          validation_split=hyper_parameters.validation_split, callbacks=[epoch_time_recorder])

Epoch 1/2
1498/1500 [============================>.] - ETA: 0s - loss: 0.5181 - accuracy: 0.8134

In [12]:
eval_dict = model.evaluate(data.X_test, data.y_test, return_dict=True)

313/313 [==============================] - 1s 2ms/step - loss: 0.3919 - accuracy: 0.8563


In [13]:
evaluator = Evaluator()
evaluator.update_evaluations("baseline_model", eval_dict, epoch_time_recorder.times)
evaluator.table

,Model Name,loss,accuracy,mean_epoch_time
0,baseline_model,0.391923,0.8563,9.175419


#### Hyperparameter Tuning

In [14]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='kt_dir',
                     project_name='kt_hyperband')

INFO:tensorflow:Reloading Tuner from kt_dir\kt_hyperband\tuner0.json


In [15]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [16]:
early_stopper = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [17]:
tuner.search(data.X_train, data.y_train, epochs=hyper_parameters.epochs, 
             validation_split=hyper_parameters.validation_split, callbacks=[early_stopper])

INFO:tensorflow:Oracle triggered exit


In [18]:
best_hps=tuner.get_best_hyperparameters()[0]
print(f"Best value for units = {best_hps.get('units')}")
print(f"Best value for learning_rate = {best_hps.get('learning_rate')}")

Best value for units = 192
Best value for learning_rate = 0.001


In [19]:
best_model: keras.Sequential = tuner.hypermodel.build(best_hps)
best_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 192)               150720    
                                                                 
 dropout_1 (Dropout)         (None, 192)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                1930      
                                                                 
Total params: 152,650
Trainable params: 152,650
Non-trainable params: 0
_________________________________________________________________


In [20]:
best_model.fit(data.X_train, data.y_train, epochs=hyper_parameters.epochs, 
          validation_split=hyper_parameters.validation_split)

Epoch 1/10
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5358 - accuracy: 0.8097 - val_loss: 0.4403 - val_accuracy: 0.8388
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4007 - accuracy: 0.8544 - val_loss: 0.3833 - val_accuracy: 0.8616
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3670 - accuracy: 0.8657 - val_loss: 0.3559 - val_accuracy: 0.8723
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3446 - accuracy: 0.8740 - val_loss: 0.3485 - val_accuracy: 0.8742
Epoch 5/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3283 - accuracy: 0.8785 - val_loss: 0.3345 - val_accuracy: 0.8766
Epoch 6/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3133 - accuracy: 0.8835 - val_loss: 0.3251 - val_accuracy: 0.8821
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3029 - accuracy: 0.8865 - val_loss: 0.3295 - val_accuracy:

In [44]:
best_eval_dict = best_model.evaluate(data.X_test, data.y_test, return_dict=True)

313/313 [==============================] - 1s 2ms/step - loss: 0.3406 - accuracy: 0.8824


In [45]:
evaluator.update_evaluations(best_eval_dict, "tuned_model")
evaluator.table


,Model Name,loss,accuracy
0,baseline_model,0.344092,0.8792
1,tuned_model,2.485904,0.0893
2,tuned_model,0.340590,0.8824


In [49]:
best_eval_dict = best_model.evaluate(data.X_train, data.y_train, return_dict=True)

1875/1875 [==============================] - 4s 2ms/step - loss: 0.2474 - accuracy: 0.9089


In [16]:
time_his = TimeRecorder()

In [17]:
his = model.fit(data.X_train, data.y_train, epochs=2, callbacks=[time_his])

Epoch 1/2
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4984 - accuracy: 0.8229
Epoch 2/2
1875/1875 [==============================] - 12s 6ms/step - loss: 0.3844 - accuracy: 0.8601


In [24]:
time_his.times

[11.999684572219849, 12.01921010017395]